# Audit effectué le 23/05/2022 sur place avec des informations complémentaires

## CREATE / ALTER TABLE

Table comprenant des notes manuelles

In [ ]:
DROP TABLE IF EXISTS `AUDIT_23_05_2022_review_2`;
CREATE TABLE `AUDIT_23_05_2022_review_2` (
  `modele` varchar(255) DEFAULT NULL,
  `Type` varchar(100) NOT NULL,
  `fabricant` varchar(255) DEFAULT NULL,
  `nom` varchar(255) DEFAULT NULL,
  `SHORT_NAME` varchar(100) NOT NULL,
  `SERIAL_NUMBER` varchar(100) NOT NULL,
  `ASSET_ID` varchar(100) NOT NULL,
  `ASSET_TAG` varchar(100) NOT NULL,
  `Asset_Number` varchar(100) NOT NULL,
  `Nom_de_la_salle` varchar(255) DEFAULT NULL,
  `reperage_id` varchar(255) DEFAULT NULL,
  `Taille_U` varchar(100) NOT NULL,
  `End_Move_Target` varchar(100) NOT NULL,
  `Salle_Dest` varchar(100) NOT NULL,
  `Rack_U_Dest` varchar(100) NOT NULL,
  `olivier_index` varchar(100) NOT NULL,
  `PROJET_Responsable` varchar(100) NOT NULL,
  `status` varchar(255) DEFAULT NULL,
  `STATUS_2` varchar(255) DEFAULT NULL,
  `Nom_de_baie` varchar(255) DEFAULT NULL,
  `Slot_u` varchar(255) DEFAULT NULL,
  `STATUS_3` varchar(255) DEFAULT NULL,
  `ASSET_ID_OK_POSITION` varchar(255) DEFAULT NULL,
    `Audit_sur_site` varchar(255) DEFAULT NULL,
    `Notes` varchar(255) DEFAULT NULL,
    `Asset_id_to_Add` varchar(255) DEFAULT NULL,
    `Source_d_information_Asset_id` varchar(255) DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=latin1;

In [ ]:
LOAD DATA LOCAL INFILE '/tmp/phpvfx7Fh' 
INTO TABLE `AUDIT_23_05_2022_review`
FIELDS TERMINATED BY ';' 
ENCLOSED BY '"'
ESCAPED BY '\\' 
LINES TERMINATED BY '\n'# 1013 rows affected.

### `Patch_Panel`

In [ ]:
DROP TABLE IF EXISTS `patch_panel_name`;
CREATE TABLE `patch_panel_name` (
  `modele` varchar(255) DEFAULT NULL ) ;

### `Linked_equipment`

In [ ]:
ALTER TABLE AUDIT_23_05_2022_review_2
ADD COLUMN link_salle varchar(255),
ADD COLUMN link_rack varchar(255),
ADD COLUMN link_asset_id varchar(255),
ADD COLUMN link_asset_tag varchar(255), 
ADD COLUMN Audit_time varchar(255),
ADD COLUMN DEFAUT varchar(255) ;

# SELECT

In [ ]:
SELECT it.asset_id asset_id_it, it.short_name, it.nom, it.status_2 status_it, it.modele, it.fabricant, it.reperage_id, it.Audit_sur_site, it.Notes, it.Asset_id_to_Add, it.Source_d_information_Asset_id
 FROM `AUDIT_23_05_2022_review_2` it
 where notes != ''

In [ ]:
SELECT it.nom, it.status_2 status_it, it.modele, it.fabricant, it.reperage_id, it.Audit_sur_site, it.Notes, it.Asset_id_to_Add, it.Source_d_information_Asset_id,
	SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' ',1) link_salle ,
	SUBSTRING_INDEX(SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' |',1),' ', -1) link_rack ,
    -- concat('M-', SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'M-',-1),'|',1)) link_asset_id,
    -- concat('AF', SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'AF',-1),'|',1)) link_tag
    
    SUBSTRING_INDEX(SUBSTRING_INDEX(notes, '|',4),'|', -1 ) link_asset_id,
    SUBSTRING_INDEX(notes, '|',-1) link_tag
    
 FROM `AUDIT_23_05_2022_review_2` it
 where notes = 'Rack : P1 C21  | computer | Appliance | M-0000657175 | HANA02-APPLIANCE_OLD | HANA02-APPLIANCE | AF209196'

# `Status is NULL`

In [ ]:
SELECT aa.modele, aa.status_3, aa.asset_id_ok_position, aa.audit_sur_site, aa.notes, aa.asset_id_to_add

FROM `AUDIT_23_05_2022_review_2` aa
where  status_3 is null and Audit_sur_site != 'patch panel'

# `Linked_equipment`

On fait le lien entre la cmdb et notes associées à une audit, généralement le bon asset_id est référencé dans la colonne asset_id_to_add

Permet de controler avant l'update de la colonne **Notes**

In [ ]:
SELECT aa.modele, aa.status_3, aa.asset_id_ok_position, aa.audit_sur_site, aa.notes, aa.asset_id_to_add ,
		concat('Rack : ',aa2.Salle, ' ', aa2.Rack, ' | ',
               aa2.ype, ' | ',
               aa2.subtype, ' | ',
               aa2.asset_id, ' | ',
               aa2.nom_court, ' | ',
               aa2.nom_long, ' | ',
               aa2.tag_partenaire, ' | ' ) agg
FROM `AUDIT_23_05_2022_review_2` aa
INNER JOIN `assets_filtree_reduced` aa2 ON aa2.asset_id = aa.Asset_id_to_Add

where aa.asset_id_to_add is not null and (aa.Notes not like '%|%' or aa.Notes is null )

In [ ]:
SELECT aa.modele, aa.status_3, aa.asset_id_ok_position, aa.audit_sur_site, aa.notes, aa.asset_id_to_add ,
		concat('Rack : ',aa2.Salle, ' ', aa2.Rack, ' | ',
               aa2.ype, ' | ',
               aa2.subtype, ' | ',
               aa2.asset_id, ' | ',
               aa2.nom_court, ' | ',
               aa2.nom_long, ' | ',
               aa2.tag_partenaire ) agg
FROM `AUDIT_23_05_2022_review_2` aa
INNER JOIN `assets_filtree_reduced` aa2 ON trim(aa2.asset_id) = trim(aa.Asset_id_to_Add)

where ASSET_ID_OK_POSITION = 'M-0000731315'

In [ ]:
SELECT * FROM `AUDIT_23_05_2022_review_2`
where Audit_sur_site not like 'patch panel' and (Nom_de_baie != link_rack or link_salle != Nom_de_la_salle) and link_rack is not null
limit 1000

## Query de base

In [ ]:
SELECT modele, fabricant, Nom_de_la_salle, reperage_id, status, STATUS_2, STATUS_3, Notes 
FROM `AUDIT_23_05_2022_review_2`

# UPDATE

## `Audit_time`

In [ ]:
UPDATE AUDIT_23_05_2022_review_2
set Audit_time = CASE WHEN STATUS_3 like 'identifiable' and ASSET_ID_OK_POSITION != '' then '20-05-2022' 
                 else Audit_time
                 END
WHERE Audit_time is null and status_2 is not null

### `Patch_Panel`

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `patch_panel_name` pp ON
pp.modele = it.modele
SET it.status_2 = 'Patch_panel'

# `Linked equipment`

On fait le lien entre la **cmdb** et notes associées à une **audit**, généralement le bon asset_id est référencé dans la colonne asset_id_to_add

In [ ]:
-- 336
UPDATE `AUDIT_23_05_2022_review_2` aa
INNER JOIN `assets_filtree_reduced` aa2 ON aa2.asset_id = aa.Asset_id_to_Add

set aa.Notes = concat('Rack : ',aa2.Salle, ' ', aa2.Rack, ' | ',
               aa2.ype, ' | ',
               aa2.subtype, ' | ',
               aa2.asset_id, ' | ',
               aa2.nom_court, ' | ',
               aa2.nom_long, ' | ',
               aa2.tag_partenaire )

where aa.asset_id_to_add is not null and (aa.Notes not like '%|%' or aa.Notes is null )

Link sur les status non null

In [ ]:
-- 376, 26-05-2022
UPDATE `AUDIT_23_05_2022_review_2` it
 SET it.link_salle    = SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' ',1),
     it.link_rack     = SUBSTRING_INDEX(SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' |',1),' ', -1),
     it.link_asset_id = SUBSTRING_INDEX(SUBSTRING_INDEX(notes, '|',4),'|', -1 ),
     it.link_asset_tag      = SUBSTRING_INDEX(notes, '|',-1) ,
     it.audit_time = '26-05-2022'
 where it.Notes like '%|%' and STATUS_2 is not null

Link sur les status null

In [ ]:
-- 376, 28-05-2022
UPDATE `AUDIT_23_05_2022_review_2` it
 SET it.link_salle    = SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' ',1),
     it.link_rack     = SUBSTRING_INDEX(SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' |',1),' ', -1),
     it.link_asset_id = SUBSTRING_INDEX(SUBSTRING_INDEX(notes, '|',4),'|', -1 ),
     it.link_asset_tag      = SUBSTRING_INDEX(notes, '|',-1) ,
     it.audit_time = '28-05-2022'
 where it.Notes like '%|%' and STATUS_2 is null

### Mauvaise salle / salle avec extension (on filtre sur P1->P4 et on trouve des P3/SB)

##### Filtre sur les 2 api et non plus sur les salles

In [ ]:
-- 23 
UPDATE `AUDIT_23_05_2022_review_2` aa
INNER JOIN `assets_filtree_v2` aa2 ON aa2.asset_id = aa.Asset_id_to_Add

set aa.Notes = concat('Rack : ',aa2.Salle, ' ', aa2.Rack, ' | ',
               aa2.ype, ' | ',
               aa2.subtype, ' | ',
               aa2.asset_id, ' | ',
               aa2.nom_court, ' | ',
               aa2.nom_long, ' | ',
               aa2.tag_partenaire ) ,
	aa.defaut = 'Wrong Salle'
where aa.asset_id_to_add is not null and (aa.Notes not like '%|%' or aa.Notes is null )

In [ ]:
-- 23, 28-05-2022
UPDATE `AUDIT_23_05_2022_review_2` it
 SET it.link_salle    = SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' ',1),
     it.link_rack     = SUBSTRING_INDEX(SUBSTRING_INDEX(SUBSTRING_INDEX(notes, 'Rack : ',-1),' |',1),' ', -1),
     it.link_asset_id = SUBSTRING_INDEX(SUBSTRING_INDEX(notes, '|',4),'|', -1 ),
     it.link_asset_tag      = SUBSTRING_INDEX(notes, '|',-1) ,
     it.audit_time = '28-05-2022'
 where it.Notes like '%|%' and it.link_rack is null

# `DEFAUT`

Ajout d'une nouvelle colonne. On essaye de retracer pour quelle raison il n'y a pas eu de match initialement

### OK-Position:

Si le status ok:position existe c'est qu'il y a plusieurs match sur une seule position. Impossible de faire le choix sans controle des bases et/ou controle physique

In [ ]:
-- 44 
UPDATE `AUDIT_23_05_2022_review_2`
set DEFAUT = "2 or more match"
where STATUS_2 like 'ok-position:%' AND DEFAUT is null

### Position incorrect

Oublie de position / mauvaise position renseigné = Recherche manuel de l'asset-id et de l'équipement

In [ ]:
-- 216
UPDATE `AUDIT_23_05_2022_review_2`
SET DEFAUT = 'Wrong position'
where (Nom_de_baie != link_rack or link_salle != Nom_de_la_salle)
and (link_rack is not null or link_rack!= '0')
and defaut is null

### Wrong salle, fait plus tôt

In [ ]:
-- 23 
UPDATE `AUDIT_23_05_2022_review_2` aa
INNER JOIN `assets_filtree_v2` aa2 ON aa2.asset_id = aa.Asset_id_to_Add

set aa.Notes = concat('Rack : ',aa2.Salle, ' ', aa2.Rack, ' | ',
               aa2.ype, ' | ',
               aa2.subtype, ' | ',
               aa2.asset_id, ' | ',
               aa2.nom_court, ' | ',
               aa2.nom_long, ' | ',
               aa2.tag_partenaire ) ,
	aa.defaut = 'Wrong Salle'
where aa.asset_id_to_add is not null and (aa.Notes not like '%|%' or aa.Notes is null )